In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
import json
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
import tensorflow as tf

# Load the validation dataset
validation_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/plant-village-dataset-updated',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(299, 299),  # Adjusting the image size to match ResNet50 input
    shuffle=True,
    interpolation="bilinear",
    subset="validation",
    validation_split=0.2,  # Split data into training and validation
    seed=42 # Provide a seed for reproducibility
)



Found 67118 files belonging to 9 classes.
Using 13423 files for validation.


In [4]:
# Load the training dataset
training_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/plant-village-dataset-updated',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(299, 299),  # Adjusting the image size to match ResNet50 input
    shuffle=True,
    interpolation="bilinear",
    subset="training",
    validation_split=0.2,  # Split data into training and validation
    seed=42  # Provide a seed for reproducibility
)


Found 67118 files belonging to 9 classes.
Using 53695 files for training.


In [5]:
# Preprocess the datasets to match ResNet50 input requirements
training_set = training_set.map(lambda x, y: (preprocess_input(x), y))
validation_set = validation_set.map(lambda x, y: (preprocess_input(x), y))

In [6]:
from tensorflow.keras.applications import InceptionV3

# Load InceptionV3 as the base model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Build the model

In [7]:
# Freeze the base model layers to prevent them from being trained
base_model.trainable = False

In [8]:
# Build the new model on top of the ResNet50 base
model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(9, activation='softmax')  # 38 classes in the dataset
])

In [9]:
from tensorflow.keras.metrics import Precision, Recall, AUC

# Compile the model with additional metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',           # Standard accuracy
        Precision(),          # Precision metric
        Recall(),             # Recall metric
        AUC(),                # AUC (Area Under Curve)
        'TruePositives',      # True positive count
        'TrueNegatives',      # True negative count
        'FalsePositives',     # False positive count
        'FalseNegatives'      # False negative count
    ]
)


In [10]:
# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)            │ (None, 8, 8, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 9)                   │           9,225 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,910,185 (91.21 MB)

 Trainable params: 2,107,401 (8.04 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [11]:
# Training the model
training_history = model.fit(
    x=training_set,
    validation_data=validation_set,
    epochs=10
)


Epoch 1/10
1678/1678 ━━━━━━━━━━━━━━━━━━━━ 214s 117ms/step - FalseNegatives: 20669.8535 - FalsePositives: 7459.8843 - TrueNegatives: 207579.9531 - TruePositives: 6210.1265 - accuracy: 0.3486 - auc: 0.7252 - loss: 4.1654 - precision: 0.4113 - recall: 0.2282 - val_FalseNegatives: 9256.0000 - val_FalsePositives: 1147.0000 - val_TrueNegatives: 106237.0000 - val_TruePositives: 4167.0000 - val_accuracy: 0.5612 - val_auc: 0.8999 - val_loss: 1.2686 - val_precision: 0.7842 - val_recall: 0.3104
Epoch 2/10
1678/1678 ━━━━━━━━━━━━━━━━━━━━ 170s 101ms/step - FalseNegatives: 17936.7109 - FalsePositives: 3945.2192 - TrueNegatives: 211094.6250 - TruePositives: 8943.2686 - accuracy: 0.5159 - auc: 0.8762 - loss: 1.3846 - precision: 0.6902 - recall: 0.3254 - val_FalseNegatives: 8069.0000 - val_FalsePositives: 1312.0000 - val_TrueNegatives: 106072.0000 - val_TruePositives: 5354.0000 - val_accuracy: 0.6230 - val_auc: 0.9253 - val_loss: 1.1049 - val_precision: 0.8032 - val_recall: 0.3989
Epoch 3/10
1678/1678 ━

In [12]:
# Evaluate the model on the training set

results = model.evaluate(training_set)

# Extracting loss and accuracy along with additional metrics
train_loss = results[0]
train_acc = results[1]
precision = results[2]
recall = results[3]
auc = results[4]
true_positives = results[5]
true_negatives = results[6]
false_positives = results[7]
false_negatives = results[8]

# Printing the results
print(f'Train Loss: {train_loss}, Train Accuracy: {train_acc}')
print(f'Precision: {precision}, Recall: {recall}, AUC: {auc}')
print(f'True Positives: {true_positives}, True Negatives: {true_negatives}')
print(f'False Positives: {false_positives}, False Negatives: {false_negatives}')

1678/1678 ━━━━━━━━━━━━━━━━━━━━ 141s 84ms/step - FalseNegatives: 10125.1719 - FalsePositives: 2429.5776 - TrueNegatives: 212610.2656 - TruePositives: 16754.8086 - accuracy: 0.7574 - auc: 0.9688 - loss: 0.7248 - precision: 0.8718 - recall: 0.6229
Train Loss: 0.7168675661087036, Train Accuracy: 0.7605736255645752
Precision: 0.877202570438385, Recall: 0.6258124709129333, AUC: 0.9696147441864014
True Positives: 33603.0, True Negatives: 424856.0
False Positives: 4704.0, False Negatives: 20092.0


In [13]:
# Evaluate the model on the validation set


results = model.evaluate(validation_set)

# Extracting loss and accuracy along with additional metrics
train_loss = results[0]
train_acc = results[1]
precision = results[2]
recall = results[3]
auc = results[4]
true_positives = results[5]
true_negatives = results[6]
false_positives = results[7]
false_negatives = results[8]

# Printing the results
print(f'Train Loss: {train_loss}, Train Accuracy: {train_acc}')
print(f'Precision: {precision}, Recall: {recall}, AUC: {auc}')
print(f'True Positives: {true_positives}, True Negatives: {true_negatives}')
print(f'False Positives: {false_positives}, False Negatives: {false_negatives}')

420/420 ━━━━━━━━━━━━━━━━━━━━ 34s 82ms/step - FalseNegatives: 2834.3540 - FalsePositives: 750.7839 - TrueNegatives: 53263.9609 - TruePositives: 3917.4893 - accuracy: 0.7135 - auc: 0.9556 - loss: 0.8392 - precision: 0.8351 - recall: 0.5752
Train Loss: 0.8186430335044861, Train Accuracy: 0.7210012674331665
Precision: 0.8403675556182861, Recall: 0.585934579372406, AUC: 0.9579842686653137
True Positives: 7865.0, True Negatives: 105890.0
False Positives: 1494.0, False Negatives: 5558.0


In [14]:
# Load the test dataset from the 'Test' subdirectories of the plant categories
test_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/plant-village-dataset-updated',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),  # Adjusting the image size to match ResNet50 input
    shuffle=False,  # Typically, you don't shuffle the test set
    interpolation="bilinear",
)

# Evaluate the model on the test set
test_results = model.evaluate(test_set)

# Extracting test metrics
test_loss = test_results[0]
test_acc = test_results[1]
test_precision = test_results[2]
test_recall = test_results[3]
test_auc = test_results[4]
test_true_positives = test_results[5]
test_true_negatives = test_results[6]
test_false_positives = test_results[7]
test_false_negatives = test_results[8]

# Printing the results for the test set
print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')
print(f'Test Precision: {test_precision}, Test Recall: {test_recall}, Test AUC: {test_auc}')
print(f'Test True Positives: {test_true_positives}, Test True Negatives: {test_true_negatives}')
print(f'Test False Positives: {test_false_positives}, Test False Negatives: {test_false_negatives}')


Found 67118 files belonging to 9 classes.
2098/2098 ━━━━━━━━━━━━━━━━━━━━ 99s 43ms/step - FalseNegatives: 27130.5938 - FalsePositives: 25136.7148 - TrueNegatives: 243663.0312 - TruePositives: 6469.3745 - accuracy: 0.1854 - auc: 0.5849 - loss: 9.6069 - precision: 0.1852 - recall: 0.1745
Test Loss: 10.730033874511719, Test Accuracy: 0.18077117204666138
Test Precision: 0.18208472430706024, Test Recall: 0.16932864487171173, Test AUC: 0.5761508345603943
Test True Positives: 11365.0, Test True Negatives: 485893.0
Test False Positives: 51051.0, Test False Negatives: 55753.0


In [17]:
# Save the model
model.save('plant-village_disease_inceptionv3.keras')

In [18]:
# Save training history to JSON
with open('hist_plant-village_disease_inceptionv3.json', 'w') as f:
    json.dump(training_history.history, f)